# Run FIR model on pared down GLM 

In [1]:
from bids.layout import BIDSLayout
from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms import modelgen as mgen
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
import nipype.utils.filemanip as fm
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html
#
# set spm path to copy with use edit of spm_fMRI_design.m [see: https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=spm;f525092b.1508]
# scratch/qbi/uqkgarn1
matlab_cmd = '/scratch/qbi/uqkgarn1/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

201229-19:42:10,708 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()

firs = pe.Workflow(name="firs") # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


# grab and sink data

In [3]:
dgT2s = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                            outfields=['func','motion','onsets','bjson','mask']), name='T2-grabber')
dgT2s.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dgT2s.inputs.sort_filelist = True
dgT2s.inputs.template='*'
dgT2s.inputs.template_args = {'func': [['sub', 'sub', 'TR']],
                              'motion':[['sub', 'sub', 'TR']],
                              'onsets':[['sub', 'sub', 'TR']],
                              'bjson':[['sub', 'sub', 'TR']],
                              'mask':[['sub', 'sub', 'TR']]}
dgT2s.inputs.field_template = {'func': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*TR%s_space-T1w_desc-preproc_bold.nii.gz',
                               'motion': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/ses-*/func/sub-%s_*-TR%s*desc-motion_regressors.txt',
                               'onsets': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*-TR%s_glm_onsets.json',
                               'bjson': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-*/func/sub-%s_*-TR%s*space-T1w_desc-preproc_bold.json',
                               'mask': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-02/func/sub-%s_*TR%s_space-T1w_desc-brain_mask.nii.gz'}

In [4]:
# # debugging
# dgT2s.inputs.sub = '01'
# dgT2s.inputs.TR = '1510'
# res = dgT2s.run()
# res.outputs

In [5]:
T2inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                   name='T2-info')
T2inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#T2inf.iterables = [('sub', ['01']), ('TR', ['1510'])]
T2inf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    fname
    l = fname.split(os.sep)
    TR = re.search('.*acq-TR([0-9]*)_.*',l[-1])
    TR = str(int(TR.group(1)))
    name = [s for s in l if re.search('sub', s)][0]
    name = [name, "TR"+TR]
    name
    name = '/'.join(name)
    return name

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# design informatiano

#### Overall

In [8]:
# the input function should be a list of the prt files, taken across sessions, for each TR
def getOnsetsJson(input_files):
    from nipype.interfaces.base import Bunch
    import json
    prt_output = [] #prt=protocol
    count = 0
    for f in input_files: 
        count = count + 1
        with open(f, "r+") as file:
            data = json.load(file)
            prt_output.insert(count, 
                              Bunch(conditions=data['names'],
                                    onsets=data['onsets'],
                                    durations=data['durations']))
    return prt_output

In [9]:
getOnsets = pe.Node(Function(input_names=['input_files'],
                             output_names=['prt_output'],
                             function=getOnsetsJson),
                    name='get_prt_onsets')

#### Target onset

In [10]:
def nu_regressors_comb2(bunches, cond_a, cond_b, nunames):
    # Warning: some elements of this function are hard coded!
    # this function will take a prt output (attained from getOnsets node),
    # and will collapse the conditions listed in cond_a, and those listed in cond_b
    # and will make 2 new conditions, labelled nunames
    from nipype.interfaces.base import Bunch
    nu_output = []
    count = 0
    for bunch in bunches:
        count = count + 1
    
        idx_a = [i for i,c in enumerate(bunch.conditions) if c in cond_a]
        idx_b = [i for i,c in enumerate(bunch.conditions) if c in cond_b]
        nuonsets = [[o  for i in idx_a for o in bunch.onsets[i]],
                    [o  for i in idx_b for o in bunch.onsets[i]]]
        nu_output.insert(count, 
                         Bunch(conditions=nunames,
                               onsets=nuonsets,
                               durations=[[0],[0]]))
    return nu_output

In [11]:
collapseOnsets = pe.Node(Function(input_names=['bunches', 'cond_a', 'cond_b', 'nunames'],
                                  output_names=['nu_output'],
                                  function=nu_regressors_comb2),
                        name='collapseOnsets')
collapseOnsets.inputs.cond_a = ['att_left_5', 'att_left_8']
collapseOnsets.inputs.cond_b = ['att_right_5', 'att_right_8']
collapseOnsets.inputs.nunames = ['tgt_left', 'tgt_right']

#### response hand

In [12]:
def select_regressors(bunches, conditions):
    # this function allows you to isolate the regressors that match the list of conditions, 
    # and returns them in the prt format for spm12
    from nipype.interfaces.base import Bunch
    nu_output = []
    count = 0
    
    for bunch in bunches:
        count = count + 1
        idx = [i for i,c in enumerate(bunch.conditions) if c in conditions]
        nuonsets = []
        nudurs = []
        for i in idx:
            nuonsets.append(bunch.onsets[i])
            nudurs.append([0])
            
        nu_output.insert(count, 
                         Bunch(conditions=conditions,
                               onsets=nuonsets,
                               durations=nudurs))
    return nu_output

In [13]:
selectOnsets = pe.Node(Function(input_names=['bunches', 'conditions'],
                                output_names=['nu_output'],
                                function=select_regressors),
                        name='selectOnsets')
selectOnsets.inputs.conditions = ['left_hand', 'right_hand']

# Gunzip nodes

In [14]:
gunzipfunc = pe.MapNode(Gunzip(), name='gunzipfunc', iterfield=['in_file'])

In [15]:
gmask = pe.Node(Gunzip(), name='m-zip')

# Get the TR for the model spec

In [16]:
# the input function should be a list of the prt files, taken across sessions, for each TR
def getTRJson(input_files):
    import json
    with open(input_files[0], "r+") as file:
            data = json.load(file)
            TR = data['RepetitionTime'] 
    if TR < .5:
        TR = 1.92
    return TR

In [17]:
getTR = pe.Node(Function(input_names=['input_files'],
                         output_names=['TR'],
                         function=getTRJson),
                name='get_TR')

In [18]:
def printSavFol(TR):
    # function to append the TR to a name for the save folder
    name = "TR" + TR
    return name

# Specify GLM Model

In [19]:
# SpecifyModel - Generates SPM-specific Model
modelspec = pe.Node(mgen.SpecifySPMModel(concatenate_runs=False,
                                         input_units='secs',
                                         output_units='secs',
                                         high_pass_filter_cutoff=128),
                    name="modelspec")

# Generate design matrix

In [20]:
# Level1Design - Generates an SPM design matrix
level1design = pe.Node(spm.Level1Design(bases={'fir': {'length': 18,
                                                       'order': 9}},
                                 timing_units='secs',
                                 model_serial_correlations='FAST'),
                       name="level1design")

# Connect workflow (collapsed onsets)

In [21]:
firs.connect([(T2inf, dgT2s, [('sub',  'sub')]), 
              (T2inf, dgT2s, [('TR',   'TR')]),
              (dgT2s, ds, [(('motion', printSubPath),
                             'container')]),
              (dgT2s, getOnsets, [('onsets', 'input_files')]),
              (getOnsets, collapseOnsets, [('prt_output', 'bunches')]),
              (dgT2s, gunzipfunc, [('func', 'in_file')]),
              (dgT2s, gmask, [('mask', 'in_file')]),
              (dgT2s, getTR, [('bjson', 'input_files')]),
              
              (getTR, modelspec, [('TR', 'time_repetition')]),
              (dgT2s, modelspec, [('motion', 'realignment_parameters')]), 
              (collapseOnsets, modelspec, [('nu_output', 'subject_info')]),
              (gunzipfunc, modelspec, [('out_file', 'functional_runs')]),
              (getTR, level1design, [('TR', 'interscan_interval')]),
              (modelspec, level1design, [('session_info', 'session_info')]),
              (gmask, level1design, [('out_file', 'mask_image')]),
              (level1design, ds, [('spm_mat_file', 'FLGLM.@des')])
])

In [22]:
firs.run()

201229-19:42:22,938 nipype.workflow INFO:
	 Workflow firs settings: ['check', 'execution', 'logging', 'monitoring']
201229-19:42:23,137 nipype.workflow INFO:
	 Running serially.
201229-19:42:23,140 nipype.workflow INFO:
	 [Node] Setting-up "firs.T2-grabber" in "/tmp/tmp6bq19mds/firs/_TR_1920_sub_05/T2-grabber".
201229-19:42:23,145 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
201229-19:42:23,170 nipype.workflow INFO:
	 [Node] Finished "firs.T2-grabber".
201229-19:42:23,172 nipype.workflow INFO:
	 [Node] Setting-up "firs.get_TR" in "/tmp/tmpjz0lk1iv/firs/_TR_1920_sub_05/get_TR".
201229-19:42:23,176 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
201229-19:42:23,181 nipype.workflow INFO:
	 [Node] Finished "firs.get_TR".
201229-19:42:23,181 nipype.workflow INFO:
	 [Node] Setting-up "firs.m-zip" in "/tmp/tmp_1vlcr6z/firs/_TR_1920_sub_05/m-zip".
201229-19:42:23,185 nipype.workflow INFO:
	 [Node] Runni

201229-19:44:30,114 nipype.workflow INFO:
	 [Node] Finished "firs.sink-stuff".
201229-19:44:30,115 nipype.workflow INFO:
	 [Node] Setting-up "firs.T2-grabber" in "/tmp/tmpqu04wicl/firs/_TR_700_sub_05/T2-grabber".
201229-19:44:30,120 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
201229-19:44:30,136 nipype.workflow INFO:
	 [Node] Finished "firs.T2-grabber".
201229-19:44:30,137 nipype.workflow INFO:
	 [Node] Setting-up "firs.get_TR" in "/tmp/tmp4tp9_90s/firs/_TR_700_sub_05/get_TR".
201229-19:44:30,142 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
201229-19:44:30,146 nipype.workflow INFO:
	 [Node] Finished "firs.get_TR".
201229-19:44:30,147 nipype.workflow INFO:
	 [Node] Setting-up "firs.m-zip" in "/tmp/tmp9r6ink5m/firs/_TR_700_sub_05/m-zip".
201229-19:44:30,151 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algorithms.misc.Gunzip")
201229-19:44:30,179 nipype.workflow INFO:
	 [Node] Finish

201229-19:46:51,678 nipype.workflow INFO:
	 [Node] Setting-up "firs.T2-grabber" in "/tmp/tmpzeoa609n/firs/_TR_1510_sub_04/T2-grabber".
201229-19:46:51,683 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
201229-19:46:51,697 nipype.workflow INFO:
	 [Node] Finished "firs.T2-grabber".
201229-19:46:51,698 nipype.workflow INFO:
	 [Node] Setting-up "firs.get_TR" in "/tmp/tmpep4hs5rj/firs/_TR_1510_sub_04/get_TR".
201229-19:46:51,702 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
201229-19:46:51,706 nipype.workflow INFO:
	 [Node] Finished "firs.get_TR".
201229-19:46:51,706 nipype.workflow INFO:
	 [Node] Setting-up "firs.m-zip" in "/tmp/tmpawnkf1n9/firs/_TR_1510_sub_04/m-zip".
201229-19:46:51,710 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algorithms.misc.Gunzip")
201229-19:46:51,749 nipype.workflow INFO:
	 [Node] Finished "firs.m-zip".
201229-19:46:51,750 nipype.workflow INFO:
	 [Node] Setting-

201229-19:49:29,212 nipype.workflow INFO:
	 [Node] Running "T2-grabber" ("nipype.interfaces.io.DataGrabber")
201229-19:49:29,231 nipype.workflow INFO:
	 [Node] Finished "firs.T2-grabber".
201229-19:49:29,232 nipype.workflow INFO:
	 [Node] Setting-up "firs.get_TR" in "/tmp/tmpg_fv53a1/firs/_TR_1920_sub_03/get_TR".
201229-19:49:29,236 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
201229-19:49:29,239 nipype.workflow INFO:
	 [Node] Finished "firs.get_TR".
201229-19:49:29,240 nipype.workflow INFO:
	 [Node] Setting-up "firs.m-zip" in "/tmp/tmpn4skkzsi/firs/_TR_1920_sub_03/m-zip".
201229-19:49:29,243 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algorithms.misc.Gunzip")
201229-19:49:29,272 nipype.workflow INFO:
	 [Node] Finished "firs.m-zip".
201229-19:49:29,273 nipype.workflow INFO:
	 [Node] Setting-up "firs.gunzipfunc" in "/tmp/tmp44sn_vqu/firs/_TR_1920_sub_03/gunzipfunc".
201229-19:49:29,278 nipype.workflow INFO:
	 [Node] Setting-

201229-19:51:21,761 nipype.workflow INFO:
	 [Node] Finished "firs.get_TR".
201229-19:51:21,762 nipype.workflow INFO:
	 [Node] Setting-up "firs.m-zip" in "/tmp/tmpt8b6swa0/firs/_TR_700_sub_03/m-zip".
201229-19:51:21,766 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algorithms.misc.Gunzip")
201229-19:51:21,802 nipype.workflow INFO:
	 [Node] Finished "firs.m-zip".
201229-19:51:21,802 nipype.workflow INFO:
	 [Node] Setting-up "firs.gunzipfunc" in "/tmp/tmpqhs9sev0/firs/_TR_700_sub_03/gunzipfunc".
201229-19:51:21,807 nipype.workflow INFO:
	 [Node] Setting-up "_gunzipfunc0" in "/tmp/tmpqhs9sev0/firs/_TR_700_sub_03/gunzipfunc/mapflow/_gunzipfunc0".
201229-19:51:21,811 nipype.workflow INFO:
	 [Node] Running "_gunzipfunc0" ("nipype.algorithms.misc.Gunzip")
201229-19:51:35,60 nipype.workflow INFO:
	 [Node] Finished "_gunzipfunc0".
201229-19:51:35,63 nipype.workflow INFO:
	 [Node] Setting-up "_gunzipfunc1" in "/tmp/tmpqhs9sev0/firs/_TR_700_sub_03/gunzipfunc/mapflow/_gunzipfunc1".
201229

201229-19:53:54,694 nipype.workflow INFO:
	 [Node] Setting-up "firs.m-zip" in "/tmp/tmptr66f9na/firs/_TR_1510_sub_02/m-zip".
201229-19:53:54,698 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algorithms.misc.Gunzip")
201229-19:53:54,745 nipype.workflow INFO:
	 [Node] Finished "firs.m-zip".
201229-19:53:54,745 nipype.workflow INFO:
	 [Node] Setting-up "firs.gunzipfunc" in "/tmp/tmpywambsa8/firs/_TR_1510_sub_02/gunzipfunc".
201229-19:53:54,750 nipype.workflow INFO:
	 [Node] Setting-up "_gunzipfunc0" in "/tmp/tmpywambsa8/firs/_TR_1510_sub_02/gunzipfunc/mapflow/_gunzipfunc0".
201229-19:53:54,753 nipype.workflow INFO:
	 [Node] Running "_gunzipfunc0" ("nipype.algorithms.misc.Gunzip")
201229-19:54:03,537 nipype.workflow INFO:
	 [Node] Finished "_gunzipfunc0".
201229-19:54:03,539 nipype.workflow INFO:
	 [Node] Setting-up "_gunzipfunc1" in "/tmp/tmpywambsa8/firs/_TR_1510_sub_02/gunzipfunc/mapflow/_gunzipfunc1".
201229-19:54:03,543 nipype.workflow INFO:
	 [Node] Running "_gunzipfunc1" (

201229-19:56:38,164 nipype.workflow INFO:
	 [Node] Running "m-zip" ("nipype.algorithms.misc.Gunzip")
201229-19:56:38,202 nipype.workflow INFO:
	 [Node] Finished "firs.m-zip".
201229-19:56:38,203 nipype.workflow INFO:
	 [Node] Setting-up "firs.gunzipfunc" in "/tmp/tmpzfpraonf/firs/_TR_1920_sub_01/gunzipfunc".
201229-19:56:38,208 nipype.workflow INFO:
	 [Node] Setting-up "_gunzipfunc0" in "/tmp/tmpzfpraonf/firs/_TR_1920_sub_01/gunzipfunc/mapflow/_gunzipfunc0".
201229-19:56:38,211 nipype.workflow INFO:
	 [Node] Running "_gunzipfunc0" ("nipype.algorithms.misc.Gunzip")
201229-19:56:44,262 nipype.workflow INFO:
	 [Node] Finished "_gunzipfunc0".
201229-19:56:44,263 nipype.workflow INFO:
	 [Node] Setting-up "_gunzipfunc1" in "/tmp/tmpzfpraonf/firs/_TR_1920_sub_01/gunzipfunc/mapflow/_gunzipfunc1".
201229-19:56:44,266 nipype.workflow INFO:
	 [Node] Running "_gunzipfunc1" ("nipype.algorithms.misc.Gunzip")
201229-19:56:50,130 nipype.workflow INFO:
	 [Node] Finished "_gunzipfunc1".
201229-19:56:50,

201229-19:58:17,225 nipype.workflow INFO:
	 [Node] Running "_gunzipfunc1" ("nipype.algorithms.misc.Gunzip")
201229-19:58:27,348 nipype.workflow INFO:
	 [Node] Finished "_gunzipfunc1".
201229-19:58:27,351 nipype.workflow INFO:
	 [Node] Finished "firs.gunzipfunc".
201229-19:58:27,351 nipype.workflow INFO:
	 [Node] Setting-up "firs.get_prt_onsets" in "/tmp/tmpfx8pl_19/firs/_TR_700_sub_01/get_prt_onsets".
201229-19:58:27,356 nipype.workflow INFO:
	 [Node] Running "get_prt_onsets" ("nipype.interfaces.utility.wrappers.Function")
201229-19:58:27,409 nipype.workflow INFO:
	 [Node] Finished "firs.get_prt_onsets".
201229-19:58:27,409 nipype.workflow INFO:
	 [Node] Setting-up "firs.collapseOnsets" in "/tmp/tmpwgr5wpan/firs/_TR_700_sub_01/collapseOnsets".
201229-19:58:27,421 nipype.workflow INFO:
	 [Node] Running "collapseOnsets" ("nipype.interfaces.utility.wrappers.Function")
201229-19:58:27,427 nipype.workflow INFO:
	 [Node] Finished "firs.collapseOnsets".
201229-19:58:27,428 nipype.workflow INF

At this point, the [regressors need to be de-orthogonalised (within condition)](https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=spm;f525092b.1508), so return to scratch and run the function redef_FIRs.m, prior to running notebook 9_run_FIR_models